In [ ]:
import batoid
import time
import numpy as np
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
telescope = batoid.Optic.fromYaml("HSC.yaml")

In [ ]:
# Get Huygens and FFT PSFs for in-focus telescope and compare
airy_radius = 1.22*750e-9/8.2*206265 # arcsec
airy_radius /= 0.168 # pixels
airy_radius *= 15 # microns

th = np.linspace(0, 2*np.pi, 100)
cth = airy_radius*np.cos(th)
sth = airy_radius*np.sin(th)

theta_x = 0.0
theta_y = 0.0
wavelength = 750e-9

fftPSFLattice = batoid.fftPSF(
    telescope, theta_x, theta_y, wavelength
)
fftpsf = fftPSFLattice.array[16:-16, 16:-16]
scale = np.sqrt(np.linalg.det(fftPSFLattice.primitiveVectors))
extent = scale*fftpsf.shape[0]/2*np.r_[-1., 1., -1., 1.] # meters
extent -= scale/2  # shift half pixel
extent *= 1e6  # meters -> microns
plt.imshow(fftpsf/fftpsf.sum(), extent=extent)
plt.plot(cth, sth, c='r', lw=1)
plt.colorbar()
plt.show()

huygensPSFLattice = batoid.huygensPSF(
    telescope, theta_x, theta_y, wavelength, nx=32
)
huygensPSF = huygensPSFLattice.array[16:-16, 16:-16]

plt.imshow(huygensPSF/huygensPSF.sum(), extent=extent)
plt.plot(cth, sth, c='r', lw=1)
plt.colorbar()
plt.show()

plt.imshow(fftpsf/fftpsf.sum() - huygensPSF/huygensPSF.sum(), extent=extent)
plt.colorbar()
plt.show()

In [ ]:
# Get Huygens and FFT PSFs for in-focus telescope and compare
airy_radius = 1.22*750e-9/8.2*206265 # arcsec
airy_radius /= 0.168 # pixels
airy_radius *= 15 # microns

th = np.linspace(0, 2*np.pi, 100)
cth = airy_radius*np.cos(th)
sth = airy_radius*np.sin(th)

theta_x = 0.0
theta_y = np.deg2rad(0.75)
wavelength = 750e-9

fftPSFLattice = batoid.fftPSF(
    telescope, theta_x, theta_y, wavelength
)
fftpsf = fftPSFLattice.array[16:-16, 16:-16]
scale = np.sqrt(np.linalg.det(fftPSFLattice.primitiveVectors))
extent = scale*fftpsf.shape[0]/2*np.r_[-1., 1., -1., 1.] # meters
extent -= scale/2  # shift half pixel
extent *= 1e6  # meters -> microns
plt.imshow(fftpsf/fftpsf.sum(), extent=extent)
plt.plot(cth, sth, c='r', lw=1)
plt.colorbar()
plt.show()

huygensPSFLattice = batoid.huygensPSF(
    telescope, theta_x, theta_y, wavelength, nx=32
)
huygensPSF = huygensPSFLattice.array[16:-16, 16:-16]

plt.imshow(huygensPSF/huygensPSF.sum(), extent=extent)
plt.plot(cth, sth, c='r', lw=1)
plt.colorbar()
plt.show()

plt.imshow(fftpsf/fftpsf.sum() - huygensPSF/huygensPSF.sum(), extent=extent)
plt.colorbar()
plt.show()

In [ ]:
# Get Huygens and FFT PSFs for out-of-focus telescope and compare
airy_radius = 1.22*750e-9/8.2*206265 # arcsec
airy_radius /= 0.168 # pixels
airy_radius *= 15 # microns

th = np.linspace(0, 2*np.pi, 100)
cth = airy_radius*np.cos(th)
sth = airy_radius*np.sin(th)

# Get Huygens and FFT PSFs for out-of-focus telescope and compare
theta_x = 0.0
theta_y = 0.0
wavelength = 750e-9
cam_dz = 200e-6

shifted_telescope = (telescope
        .withGloballyShiftedOptic("SubaruHSC.HSC", [0, 0, cam_dz])
)
nx = 256

fftPSFLattice = batoid.fftPSF(
    shifted_telescope, theta_x, theta_y, wavelength, nx=nx
)
fftpsf = fftPSFLattice.array[nx//2:-nx//2, nx//2:-nx//2]
scale = np.sqrt(np.linalg.det(fftPSFLattice.primitiveVectors))
extent = scale*fftpsf.shape[0]/2*np.r_[-1., 1., -1., 1.] # meters
extent -= scale/2  # shift half pixel
extent *= 1e6  # meters -> microns
plt.imshow(fftpsf/fftpsf.sum(), extent=extent)
plt.plot(cth, sth, c='r', lw=1)
plt.colorbar()
plt.show()

huygensPSFLattice = batoid.huygensPSF(
    shifted_telescope, theta_x, theta_y, wavelength, nx=nx, nxOut=nx//2
)
huygensPSF = huygensPSFLattice.array

plt.imshow(huygensPSF/huygensPSF.sum(), extent=extent)
plt.plot(cth, sth, c='r', lw=1)
plt.colorbar()
plt.show()

plt.imshow(fftpsf/fftpsf.sum() - huygensPSF/huygensPSF.sum(), extent=extent)
plt.colorbar()
plt.show()